In [4]:
"""
Avi Shah, 11/14/2022. Yang Lab NJMS

This script collects counts produced by featureCounts
and also produces fpkm.
"""

import os
import pandas as pd

save_dir = os.path.join("/scratch", "as2654", "RNAseq0")
os.chdir(save_dir)
print(len(os.listdir(".")))
print(len([k for k in os.listdir(".") if k.endswith(".counts.txt")]))

478
72


In [5]:
# get sample names and organism from index.csv

import math
import csv

# cross reference the index and the data downloaded from
# Illumina's basespace to make sure all the files are there
# and enforce that the names should match

# first i will collect sample names

sample_mdmap = dict()

# save all the sample names and the organism they came from
with open("index.csv") as csvf:
    reader = csv.reader(csvf, delimiter=",")
    for i, line in enumerate(reader):
        #print('line[{}] = {}'.format(i, line))
        orgstr = ""
        if line[4].startswith("M. tuberculosis"):
            orgstr = "mtb"            
        elif line[4].startswith("E. coli"):
            orgstr = "ecoli"
        #print(line[1], orgstr)
        if orgstr:
            #print(i)
            sample_mdmap[line[1]] = orgstr

for sample_name, organism in sample_mdmap.items():
    print(sample_name, organism)

print(len(sample_mdmap.items()))


CTL1 mtb
CTL2 mtb
CTL3 mtb
I2 mtb
I3 mtb
IB1 mtb
IB2 mtb
IB3 mtb
IR1 mtb
IR2 mtb
IR3 mtb
IC1 mtb
IC2 mtb
IC3 mtb
IQ1 mtb
IQ2 mtb
IQ3 mtb
RV1 mtb
RV2 mtb
RV3 mtb
RV4 mtb
RV5 mtb
RV6 mtb
KG1 mtb
KG2 mtb
KG3 mtb
KG4 mtb
KG5 mtb
KG6 mtb
WT1-BL ecoli
WT2-BL ecoli
WT3-BL ecoli
atpA1-BL ecoli
atpA2-BL ecoli
atpA3-BL ecoli
I1 mtb
36


In [11]:
# compile counts

import numpy as np

runs = [fn for fn in os.listdir(".") if fn.endswith(".counts.txt")]

def get_sample_name(filename):
    return filename.replace(".counts.txt", "")

def get_sample_number(filename):
    return int(filename.split("_")[1][1:])

def get_lane(filename):
    return filename.split("_")[2]

runs.sort(key=get_lane)
runs.sort(key=get_sample_number)

print(len(runs))
for run in runs:
    print(run)
    

72
CTL1_S1_L001.counts.txt
CTL1_S1_L002.counts.txt
CTL2_S2_L001.counts.txt
CTL2_S2_L002.counts.txt
CTL3_S3_L001.counts.txt
CTL3_S3_L002.counts.txt
I2_S4_L001.counts.txt
I2_S4_L002.counts.txt
I3_S5_L001.counts.txt
I3_S5_L002.counts.txt
IB1_S6_L001.counts.txt
IB1_S6_L002.counts.txt
IB2_S7_L001.counts.txt
IB2_S7_L002.counts.txt
IB3_S8_L001.counts.txt
IB3_S8_L002.counts.txt
IR1_S9_L001.counts.txt
IR1_S9_L002.counts.txt
IR2_S10_L001.counts.txt
IR2_S10_L002.counts.txt
IR3_S11_L001.counts.txt
IR3_S11_L002.counts.txt
IC1_S12_L001.counts.txt
IC1_S12_L002.counts.txt
IC2_S13_L001.counts.txt
IC2_S13_L002.counts.txt
IC3_S14_L001.counts.txt
IC3_S14_L002.counts.txt
IQ1_S15_L001.counts.txt
IQ1_S15_L002.counts.txt
IQ2_S16_L001.counts.txt
IQ2_S16_L002.counts.txt
IQ3_S17_L001.counts.txt
IQ3_S17_L002.counts.txt
RV1_S18_L001.counts.txt
RV1_S18_L002.counts.txt
RV2_S19_L001.counts.txt
RV2_S19_L002.counts.txt
RV3_S20_L001.counts.txt
RV3_S20_L002.counts.txt
RV4_S21_L001.counts.txt
RV4_S21_L002.counts.txt
RV5_S

In [20]:
import pandas as pd

runs_mtb = []
runs_ecoli = []

for run in runs:
    name = get_sample_name(run).split("_")[0]
    if sample_mdmap[name] == "mtb":
        runs_mtb.append(run)
    elif sample_mdmap[name] == "ecoli":
        runs_ecoli.append(run)


df_mtb = pd.read_csv(runs_mtb[0], header=1, sep="\t")
df_mtb.rename(columns={df_mtb.columns[-1]: df_mtb.columns[-1].split(".")[0]}, inplace=True)
for run in runs_mtb[1:]:
    df_add = pd.read_csv(run, header=1, sep="\t")
    df_add.rename(columns={df_add.columns[-1]: df_add.columns[-1].split(".")[0]}, inplace=True)
    df_mtb[df_add.columns[-1]] = df_add[df_add.columns[-1]]

df_ecoli =  pd.read_csv(runs_ecoli[0], header=1, sep="\t")
df_ecoli.rename(columns={df_ecoli.columns[-1]: df_ecoli.columns[-1].split(".")[0]}, inplace=True)
for run in runs_ecoli[1:]:
    df_add = pd.read_csv(run, header=1, sep="\t")
    df_add.rename(columns={df_add.columns[-1]: df_add.columns[-1].split(".")[0]}, inplace=True)
    df_ecoli[df_add.columns[-1]] = df_add[df_add.columns[-1]]



In [24]:
# replace gene-AllandRv_# with gene name or Rv#

# WILL DO THIS LATER

print(df_mtb)

                    Geneid           Chr    Start      End Strand  Length  \
0     gene-AllandRv_000001  H37Rv_Alland        1     1524      +    1524   
1     gene-AllandRv_000002  H37Rv_Alland     2052     3260      +    1209   
2     gene-AllandRv_000003  H37Rv_Alland     3280     4437      +    1158   
3     gene-AllandRv_000004  H37Rv_Alland     4434     4997      +     564   
4     gene-AllandRv_000005  H37Rv_Alland     5240     7267      +    2028   
5     gene-AllandRv_000006  H37Rv_Alland     7302     9818      +    2517   
6     gene-AllandRv_000007  H37Rv_Alland     9914    10828      +     915   
7     gene-AllandRv_000008  H37Rv_Alland    10887    10960      +      74   
8     gene-AllandRv_000009  H37Rv_Alland    11112    11184      +      73   
9     gene-AllandRv_000010  H37Rv_Alland    11555    11692      -     138   
10    gene-AllandRv_000011  H37Rv_Alland    11874    12311      -     438   
11    gene-AllandRv_000012  H37Rv_Alland    12468    13016      +     549   

In [23]:
# get tpm and log2 tpm
# this method accepts counts with the first 6 columns
# as output by featureCounts intact
def get_tpms(df_out):
    tpm = df_out.copy(deep=True)
    log2tpm = df_out.copy(deep=True)
    
    for colindex in range(6,tpm.shape[1]):
        tpm.iloc[:,colindex] = tpm.iloc[:,colindex].divide(tpm['Length'])
        tpmf = sum(tpm.iloc[:,colindex]) / 1e6
        tpm.iloc[:,colindex] = tpm.iloc[:,colindex].divide(tpmf)
        log2tpm.iloc[:,colindex] = np.log2(tpm.iloc[:,colindex] + 1)

    tpm = tpm.set_index(tpm["Geneid"])
    tpm = tpm.drop(columns = tpm.columns[:6])
    
    log2tpm = log2tpm.set_index(log2tpm["Geneid"])
    log2tpm = log2tpm.drop(columns = log2tpm.columns[:6])
    
    return tpm, log2tpm


In [25]:
# save to ipynb dir
os.chdir(os.path.join("/home", "as2654", "Yang", "RNAseq0"))

mtb_tpm, mtb_log2tpm = get_tpms(df_mtb)

idstr = "M.tubercolosis"
mtb_tpm.to_csv(idstr + "_tpm.csv", index=True, header=True, sep=",")
mtb_log2tpm.to_csv(idstr + "_log2tpm.csv", index=True, header=True, sep=",")
df_mtb.to_csv(idstr + "_counts.csv", index=False, header=True, sep=",")


ecoli_tpm, ecoli_log2tpm = get_tpms(df_ecoli)

idstr = "E.coli"
ecoli_tpm.to_csv(idstr + "_tpm.csv", index=True, header=True, sep=",")
ecoli_log2tpm.to_csv(idstr + "_log2tpm.csv", index=True, header=True, sep=",")
df_ecoli.to_csv(idstr + "_counts.csv", index=False, header=True, sep=",")